In [5]:
# old works
import torch
import lpips
from PIL import Image
from torchvision import transforms
from clip import load

# Load CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, preprocess = load("ViT-B/32", device=device)

# Load LPIPS loss model
lpips_loss = lpips.LPIPS(net="vgg").to(device)

# Load images
def load_image(filepath):
    img = Image.open(filepath).convert("RGB")
    transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])
    return transform(img).unsqueeze(0).to(device)

cat_image = load_image("cat.webp")
dog_image = load_image("dog.webp")

# Use a copy of the cat image as the starting point for poisoning
poisoned_image = cat_image.clone().detach().requires_grad_(True)

# Define optimizer and hyperparameters
optimizer = torch.optim.Adam([poisoned_image], lr=0.01)
iterations = 1500  # Increased for more refinement
lpips_weight = 1.0  # Weight for perceptual similarity loss
clip_weight = 10.0  # Weight for CLIP similarity to "a dog"

# Target text embeddings
text_cat = clip_model.encode_text(clip.tokenize(["a cat"]).to(device)).detach()
text_dog = clip_model.encode_text(clip.tokenize(["a dog"]).to(device)).detach()

# Optimization loop
for i in range(iterations):
    optimizer.zero_grad()

    # Calculate LPIPS loss (perceptual similarity to cat)
    lpips_value = lpips_loss(poisoned_image, cat_image)

    # Calculate CLIP similarity scores
    image_features = clip_model.encode_image(poisoned_image)
    clip_sim_to_dog = torch.cosine_similarity(image_features, text_dog, dim=1)
    clip_sim_to_cat = torch.cosine_similarity(image_features, text_cat, dim=1)

    # Loss: increase similarity to "a dog" while staying perceptually close to "a cat"
    loss = (
        lpips_weight * lpips_value.mean()
        - clip_weight * clip_sim_to_dog.mean()
        + clip_weight * clip_sim_to_cat.mean()
    )

    loss.backward()
    optimizer.step()

    # Clamp pixel values to valid range [0, 1]
    poisoned_image.data = torch.clamp(poisoned_image.data, 0, 1)

    if i % 100 == 0 or i == iterations - 1:
        print(f"Iteration {i}/{iterations} - Loss: {loss.item()}")

# Save the final poisoned image
poisoned_image_tensor = poisoned_image.detach().squeeze().permute(1, 2, 0).cpu().numpy()
poisoned_image_pil = Image.fromarray((poisoned_image_tensor * 255).astype("uint8"))
poisoned_image_pil.save("poisoned_image_refined.png")
print("Poisoned image saved as poisoned_image_refined.png")




Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]
Loading model from: c:\Users\mfaizanz\AppData\Local\Programs\Python\Python312\Lib\site-packages\lpips\weights\v0.1\vgg.pth
Iteration 0/1500 - Loss: 0.470703125
Iteration 100/1500 - Loss: -3.1037638187408447
Iteration 200/1500 - Loss: -3.272459030151367
Iteration 300/1500 - Loss: -3.3146493434906006
Iteration 400/1500 - Loss: -3.3419196605682373
Iteration 500/1500 - Loss: -3.3389148712158203
Iteration 600/1500 - Loss: -3.3524906635284424
Iteration 700/1500 - Loss: -3.3675856590270996
Iteration 800/1500 - Loss: -3.362954616546631
Iteration 900/1500 - Loss: -3.366569995880127
Iteration 1000/1500 - Loss: -3.3886606693267822
Iteration 1100/1500 - Loss: -3.372663974761963
Iteration 1200/1500 - Loss: -3.3955347537994385
Iteration 1300/1500 - Loss: -3.2939345836639404
Iteration 1400/1500 - Loss: -3.4009644985198975
Iteration 1499/1500 - Loss: -3.39396333694458
Poisoned image saved as poisoned_image_refined.png


In [7]:
import torch
import lpips
from PIL import Image
from torchvision import transforms
from clip import load

# Load CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, preprocess = load("ViT-B/32", device=device)

# Load LPIPS loss model
lpips_loss = lpips.LPIPS(net="vgg").to(device)

# Function to load and preprocess images
def load_image(filepath):
    img = Image.open(filepath).convert("RGB")
    transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])
    return transform(img).unsqueeze(0).to(device)

# Load images
cat_image = load_image("cat.webp")
dog_image = load_image("dog.webp")

# Use a copy of the dog image as the starting point for poisoning
poisoned_image = dog_image.clone().detach().requires_grad_(True)

# Define optimizer and hyperparameters
optimizer = torch.optim.Adam([poisoned_image], lr=0.01)
iterations = 1500  # Number of optimization steps
lpips_weight = 1.0  # Weight for perceptual similarity loss
clip_weight = 10.0  # Weight for CLIP similarity to "a cat"

# Target text embeddings
text_cat = clip_model.encode_text(clip.tokenize(["a cat"]).to(device)).detach()
text_dog = clip_model.encode_text(clip.tokenize(["a dog"]).to(device)).detach()

# Optimization loop
for i in range(iterations):
    optimizer.zero_grad()

    # Calculate LPIPS loss (perceptual similarity to dog)
    lpips_value = lpips_loss(poisoned_image, dog_image)

    # Calculate CLIP similarity scores
    image_features = clip_model.encode_image(poisoned_image)
    clip_sim_to_cat = torch.cosine_similarity(image_features, text_cat, dim=1)
    clip_sim_to_dog = torch.cosine_similarity(image_features, text_dog, dim=1)

    # Loss: increase similarity to "a cat" while staying perceptually close to "a dog"
    loss = (
        lpips_weight * lpips_value.mean()
        - clip_weight * clip_sim_to_cat.mean()
        + clip_weight * clip_sim_to_dog.mean()
    )

    loss.backward()
    optimizer.step()

    # Clamp pixel values to valid range [0, 1]
    poisoned_image.data = torch.clamp(poisoned_image.data, 0, 1)

    # Print progress
    if i % 100 == 0 or i == iterations - 1:
        print(f"Iteration {i}/{iterations} - Loss: {loss.item()}")

# Save the final poisoned image
poisoned_image_tensor = poisoned_image.detach().squeeze().permute(1, 2, 0).cpu().numpy()
poisoned_image_pil = Image.fromarray((poisoned_image_tensor * 255).astype("uint8"))
poisoned_image_pil.save("dog_poisoned_image.png")
print("Poisoned image saved as dog_poisoned_image.png")


Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]
Loading model from: c:\Users\mfaizanz\AppData\Local\Programs\Python\Python312\Lib\site-packages\lpips\weights\v0.1\vgg.pth
Iteration 0/1500 - Loss: 0.59765625
Iteration 100/1500 - Loss: -2.829263210296631
Iteration 200/1500 - Loss: -3.087207794189453
Iteration 300/1500 - Loss: -3.1105921268463135
Iteration 400/1500 - Loss: -3.1543965339660645
Iteration 500/1500 - Loss: -3.1767537593841553
Iteration 600/1500 - Loss: -3.217648506164551
Iteration 700/1500 - Loss: -3.2401742935180664
Iteration 800/1500 - Loss: -3.239015817642212
Iteration 900/1500 - Loss: -3.2649776935577393
Iteration 1000/1500 - Loss: -3.24422550201416
Iteration 1100/1500 - Loss: -3.2723782062530518
Iteration 1200/1500 - Loss: -3.2668821811676025
Iteration 1300/1500 - Loss: -3.2586886882781982
Iteration 1400/1500 - Loss: -3.2438607215881348
Iteration 1499/1500 - Loss: -3.2828564643859863
Poisoned image saved as dog_poisoned_image.png


In [2]:
import torch
import clip
from PIL import Image

# Load the CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# Load the images and preprocess them
cat_image = preprocess(Image.open("cat.webp").convert("RGB")).unsqueeze(0).to(device)
dog_image = preprocess(Image.open("dog.webp").convert("RGB")).unsqueeze(0).to(device)
poisoned_image = preprocess(Image.open("dog-nightshade-intensity-DEFAULT-V1.png").convert("RGB")).unsqueeze(0).to(device)

# Define a list of possible text labels
text_labels = ["a cat", "a dog", "a car", "a tree", "a house"]
text_inputs = clip.tokenize(text_labels).to(device)

# Encode the images and text using CLIP
with torch.no_grad():
    cat_features = model.encode_image(cat_image)
    dog_features = model.encode_image(dog_image)
    poisoned_features = model.encode_image(poisoned_image)
    text_features = model.encode_text(text_inputs)

# Normalize the features for cosine similarity
cat_features /= cat_features.norm(dim=-1, keepdim=True)
dog_features /= dog_features.norm(dim=-1, keepdim=True)
poisoned_features /= poisoned_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)

# Compute similarity scores
cat_similarity = (cat_features @ text_features.T).squeeze()
dog_similarity = (dog_features @ text_features.T).squeeze()
poisoned_similarity = (poisoned_features @ text_features.T).squeeze()

# Print similarity scores for each label
print("Similarity scores for cat.webp:")
for i, label in enumerate(text_labels):
    print(f"  {label}: {cat_similarity[i].item():.4f}")

print("\nSimilarity scores for dog.webp:")
for i, label in enumerate(text_labels):
    print(f"  {label}: {dog_similarity[i].item():.4f}")

print("\nSimilarity scores for poisoned_image_refined.png:")
for i, label in enumerate(text_labels):
    print(f"  {label}: {poisoned_similarity[i].item():.4f}")


Similarity scores for cat.webp:
  a cat: 0.2534
  a dog: 0.2063
  a car: 0.1832
  a tree: 0.1753
  a house: 0.1829

Similarity scores for dog.webp:
  a cat: 0.1954
  a dog: 0.2588
  a car: 0.1805
  a tree: 0.1611
  a house: 0.1715

Similarity scores for poisoned_image_refined.png:
  a cat: 0.1962
  a dog: 0.2600
  a car: 0.1809
  a tree: 0.1617
  a house: 0.1705
